<a target="_blank" href="https://colab.research.google.com/github/vnstock-hq/vnstock_insider_guide/blob/main/demo/4-vnstock_news_v2.1_demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# I. Cài đặt thư viện

In [ ]:
# Sao chép mã nguồn từ GitHub về Google Colab
!git clone https://github.com/vnstock-hq/vnstock_insider_guide
# Chuyển đến thư mục vừa clone
%cd /content/vnstock_insider_guide/oneclick_installer
# Trao quyền thực thi cho file
!chmod +x linux_installer.run
# Chạy trình cài đặt
!./linux_installer.run

**Lưu ý: Bạn cần khởi động lại phiên làm việc sau khi cài đặt trên Google Colab**

![restar session](http://vnstocks.com/images/khoi-dong-lai-phien-lam-viec-colab.png)

# II. Minh họa

## Các trang hỗ trợ

In [ ]:
from vnstock_news import list_supported_sites
list_supported_sites()

2025-08-31 15:56:21,300 - Cache - INFO - Initialized memory cache system


DANH SÁCH CÁC TRANG WEB ĐƯỢC HỖ TRỢ:
Tên viết tắt    | Tên miền
--------------------------------------------------
cafef           | cafef.vn
cafebiz         | cafebiz.vn
vietstock       | vietstock.vn
vneconomy       | vneconomy.vn
plo             | plo.vn
vnexpress       | vnexpress.net
tuoitre         | tuoitre.vn
ktsg            | thesaigontimes.vn
ncdt            | nhipcaudautu.vn
dddn            | diendandoanhnghiep.vn
baodautu        | baodautu.vn


[{'name': 'cafef', 'domain': 'cafef.vn'},
 {'name': 'cafebiz', 'domain': 'cafebiz.vn'},
 {'name': 'vietstock', 'domain': 'vietstock.vn'},
 {'name': 'vneconomy', 'domain': 'vneconomy.vn'},
 {'name': 'plo', 'domain': 'plo.vn'},
 {'name': 'vnexpress', 'domain': 'vnexpress.net'},
 {'name': 'tuoitre', 'domain': 'tuoitre.vn'},
 {'name': 'ktsg', 'domain': 'thesaigontimes.vn'},
 {'name': 'ncdt', 'domain': 'nhipcaudautu.vn'},
 {'name': 'dddn', 'domain': 'diendandoanhnghiep.vn'},
 {'name': 'baodautu', 'domain': 'baodautu.vn'}]

## Chương trình mẫu

In [ ]:
# Chuyển thư mục làm việc về nơi bạn muốn lưu dữ liệu tin tức
%cd /content
!vnstock-news-crawl

### Cập nhật tin tức từ 1 trang cụ thể

#### Cập nhật tin tóm tắt từ Datafeed

In [4]:
import os
import pandas as pd
import logging
import asyncio
from vnstock_news.async_crawlers.async_batch import AsyncBatchCrawler
from vnstock_news.config.sites import SITES_CONFIG
from vnstock_news.utils.logger import setup_logger
from vnstock_news.config.sitemap_resolver import DynamicSitemapResolver

# Khởi tạo logger
logger = setup_logger("export_feed_info", debug=True)

# Thư mục để lưu file CSV
OUTPUT_DIR = "output/feed_info"

async def fetch_feed_info(site_name: str, top_n: int = None, within: str = None):
    """
    Thu thập thông tin tóm tắt từ sitemap hoặc RSS feed và lưu vào file CSV.

    Parameters:
        site_name (str): Tên của trang (từ SITES_CONFIG).
        top_n (int): Giới hạn số lượng bài viết cần lấy.
        within (str): Khoảng thời gian để lọc bài viết (ví dụ: '1h', '1d', '7d').
    """
    try:
        logger.info(f"Đang xử lý trang: {site_name}")
        
        # Khởi tạo AsyncBatchCrawler
        crawler = AsyncBatchCrawler(site_name=site_name)

        # Xác định nguồn (sitemap hoặc RSS)
        site_data = SITES_CONFIG.get(site_name)
        sources = []
        if site_data.get("rss"):
            sources = site_data["rss"]["urls"]
        elif "sitemap" in site_data: # Xử lý cấu trúc sitemap mới
            resolver = DynamicSitemapResolver()
            sitemap_url = resolver.resolve_sitemap_url(site_name, site_data)
            if sitemap_url:
                sources = [sitemap_url]
        elif site_data.get("sitemap_url"): # Cấu trúc cũ
            sources = [site_data["sitemap_url"]]

        if not sources:
            logger.error(f"Không tìm thấy nguồn cho trang {site_name}. Bỏ qua...")
            return

        # Thu thập dữ liệu feed (không crawl chi tiết bài viết)
        logger.info(f"Đang thu thập thông tin feed từ các nguồn: {sources}")
        feed_df = await crawler.prepare_feeder_async(sources)

        # Lọc dữ liệu nếu có điều kiện
        if not feed_df.empty:
            filtered_df = crawler.filter_feeder(feed_df, top_n=top_n, within=within)
        else:
            logger.warning(f"Không có dữ liệu nào được thu thập từ {site_name}")
            return

        # Thiết lập thư mục output
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        output_file = os.path.join(OUTPUT_DIR, f"{site_name}_feed_info.csv")

        # Lưu vào file CSV
        if not filtered_df.empty:
            filtered_df.to_csv(output_file, index=False, encoding="utf-8-sig")
            logger.info(f"Đã lưu {len(filtered_df)} bản ghi vào {output_file}")
        else:
            logger.warning(f"Không có bản ghi nào sau khi lọc cho {site_name}")

    except Exception as e:
        logger.error(f"Lỗi khi xử lý trang '{site_name}': {e}")


async def main():
    """
    Hàm chính để thu thập và lưu thông tin feed.
    """
    logger.info("Bắt đầu quá trình xuất thông tin feed...")

    await fetch_feed_info(site_name='vnexpress', top_n=20, within="1d")  # Cập nhật theo yêu cầu

    logger.info("Quá trình xuất thông tin feed đã hoàn tất!")

# --- Cách chạy trong Jupyter Notebook ---
# Dán đoạn code này vào một cell và chạy
await main()

2025-08-31 15:42:58,923 - export_feed_info - INFO - Bắt đầu quá trình xuất thông tin feed...
2025-08-31 15:42:58,924 - export_feed_info - INFO - Đang xử lý trang: vnexpress
2025-08-31 15:42:58,925 - export_feed_info - INFO - Đang thu thập thông tin feed từ các nguồn: ['https://vnexpress.net/rss/tin-moi-nhat.rss']
2025-08-31 15:42:58,927 - AsyncBatchCrawler - INFO - Checking source: https://vnexpress.net/rss/tin-moi-nhat.rss
2025-08-31 15:42:58,927 - AsyncBatchCrawler - INFO - Parsing RSS feed: https://vnexpress.net/rss/tin-moi-nhat.rss
2025-08-31 15:42:59,030 - export_feed_info - INFO - Đã lưu 20 bản ghi vào output/feed_info/vnexpress_feed_info.csv
2025-08-31 15:42:59,031 - export_feed_info - INFO - Quá trình xuất thông tin feed đã hoàn tất!


#### Cập nhật tin chi tiết từ trang cụ thể

In [ ]:
import os
import pandas as pd
import logging
import asyncio
from vnstock_news.async_crawlers.async_batch import AsyncBatchCrawler
from vnstock_news.config.sites import SITES_CONFIG
from vnstock_news.utils.logger import setup_logger
from vnstock_news.config.sitemap_resolver import DynamicSitemapResolver

# Khởi tạo logger
logger = setup_logger("update_news", debug=False)

# Thư mục để lưu file CSV
OUTPUT_DIR = "output/cafebiz"

async def fetch_and_save_news(site_name: str, top_n: int = 10, within: str = None):
    """
    Thu thập bài viết từ một trang cụ thể và lưu vào file CSV.

    Parameters:
        site_name (str): Tên của trang (từ SITES_CONFIG).
        top_n (int): Số lượng bài viết cần lấy.
        within (str): Khoảng thời gian để lọc bài viết (ví dụ: '1h', '2d').
    """
    try:
        logger.info(f"Đang xử lý trang: {site_name}")
        
        # Khởi tạo AsyncBatchCrawler
        crawler = AsyncBatchCrawler(site_name=site_name)

        # Xác định nguồn (sitemap hoặc RSS)
        site_data = SITES_CONFIG.get(site_name)
        sources = []
        if site_data.get("rss"):
            sources = site_data["rss"]["urls"]
        elif "sitemap" in site_data: # Xử lý cấu trúc sitemap mới
            resolver = DynamicSitemapResolver()
            sitemap_url = resolver.resolve_sitemap_url(site_name, site_data)
            if sitemap_url:
                sources = [sitemap_url]
        elif site_data.get("sitemap_url"): # Cấu trúc cũ
            sources = [site_data["sitemap_url"]]

        if not sources:
            logger.error(f"Không tìm thấy nguồn cho trang {site_name}. Bỏ qua...")
            return

        # Thu thập bài viết
        logger.info(f"Đang thu thập bài viết từ các nguồn: {sources}")
        articles_df = await crawler.fetch_articles_async(sources, top_n=top_n, within=within)

        # Thiết lập thư mục output
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        output_file = os.path.join(OUTPUT_DIR, f"{site_name}_news.csv")

        # Lưu vào file CSV
        if not articles_df.empty:
            articles_df.to_csv(output_file, index=False, encoding="utf-8-sig")
            logger.info(f"Đã lưu {len(articles_df)} bài viết vào {output_file}")
        else:
            logger.warning(f"Không có bài viết nào được thu thập từ {site_name}")

    except Exception as e:
        logger.error(f"Lỗi khi xử lý trang '{site_name}': {e}")


async def main():
    """
    Hàm chính để thu thập và lưu tin tức.
    """
    logger.info("Bắt đầu quá trình cập nhật tin tức...")

    # Thay thế tên trang báo cần truy xuất dữ liệu
    await fetch_and_save_news(site_name='cafebiz', top_n=100, within="1h")

    logger.info("Quá trình cập nhật tin tức đã hoàn tất!")

# --- Cách chạy trong Jupyter Notebook ---
# Trong một cell của Jupyter, bạn chỉ cần gọi hàm main với await như sau:
await main()

2025-08-31 15:41:46,532 - update_news - INFO - Bắt đầu quá trình cập nhật tin tức...
2025-08-31 15:41:46,533 - update_news - INFO - Đang xử lý trang: cafebiz
2025-08-31 15:41:46,533 - update_news - INFO - Đang thu thập bài viết từ các nguồn: ['https://cafebiz.vn/rss/home.rss', 'https://cafebiz.vn/rss/cau-chuyen-kinh-doanh.rss', 'https://cafebiz.vn/rss/vi-mo.rss', 'https://cafebiz.vn/rss/cong-nghe.rss', 'https://cafebiz.vn/rss/xa-hoi.rss']
2025-08-31 15:41:46,534 - AsyncBatchCrawler - INFO - Checking source: https://cafebiz.vn/rss/home.rss
2025-08-31 15:41:46,534 - AsyncBatchCrawler - INFO - Parsing RSS feed: https://cafebiz.vn/rss/home.rss
2025-08-31 15:41:46,536 - RSS - INFO - Fetching RSS feed: https://cafebiz.vn/rss/home.rss
2025-08-31 15:41:46,536 - AsyncBatchCrawler - INFO - Checking source: https://cafebiz.vn/rss/cau-chuyen-kinh-doanh.rss
2025-08-31 15:41:46,537 - AsyncBatchCrawler - INFO - Parsing RSS feed: https://cafebiz.vn/rss/cau-chuyen-kinh-doanh.rss
2025-08-31 15:41:46,538

2025-08-31 15:42:58,928 - RSS - INFO - Fetching RSS feed: https://vnexpress.net/rss/tin-moi-nhat.rss
2025-08-31 15:44:20,131 - Sitemap - INFO - Fetching sitemap from https://cafef.vn/latest-news-sitemap.xml
2025-08-31 15:44:20,134 - RSS - INFO - Fetching RSS feed: https://cafebiz.vn/rss/home.rss
2025-08-31 15:44:20,136 - RSS - INFO - Fetching RSS feed: https://cafebiz.vn/rss/cau-chuyen-kinh-doanh.rss
2025-08-31 15:44:20,139 - RSS - INFO - Fetching RSS feed: https://cafebiz.vn/rss/vi-mo.rss
2025-08-31 15:44:20,145 - RSS - INFO - Fetching RSS feed: https://cafebiz.vn/rss/cong-nghe.rss
2025-08-31 15:44:20,407 - RSS - INFO - Fetching RSS feed: https://tuoitre.vn/rss/thoi-su.rss
2025-08-31 15:44:21,349 - Sitemap - WARNING - No URLs found in sitemap.
2025-08-31 15:44:21,350 - Sitemap - ERROR - Failed to parse sitemap manually: "None of [Index(['url'], dtype='object')] are in the [columns]"
2025-08-31 15:45:20,452 - Sitemap - INFO - Fetching sitemap from https://cafef.vn/latest-news-sitemap.x

### Cập nhật tin tức từ nhiều trang cùng lúc

#### Cập nhật tóm tắt từ Datafeed

In [ ]:
import os
import pandas as pd
import logging
import asyncio
from vnstock_news.async_crawlers.async_batch import AsyncBatchCrawler
from vnstock_news.config.sites import SITES_CONFIG
from vnstock_news.utils.logger import setup_logger
from vnstock_news.config.sitemap_resolver import DynamicSitemapResolver

# Khởi tạo logger
logger = setup_logger("export_feed_info", debug=False)

# Thư mục để lưu file CSV
OUTPUT_DIR = "output/feed_info"

async def fetch_feed_info(site_name: str, top_n: int = None, within: str = None):
    """
    Thu thập thông tin tóm tắt từ sitemap hoặc RSS feed và lưu vào file CSV.

    Parameters:
        site_name (str): Tên của trang (từ SITES_CONFIG).
        top_n (int): Giới hạn số lượng bài viết cần lấy.
        within (str): Khoảng thời gian để lọc bài viết (ví dụ: '1h', '1d', '7d').
    """
    try:
        logger.info(f"Đang xử lý trang: {site_name}")
        
        # Khởi tạo AsyncBatchCrawler
        crawler = AsyncBatchCrawler(site_name=site_name)

        # Xác định nguồn (sitemap hoặc RSS)
        site_data = SITES_CONFIG.get(site_name)
        sources = []
        if site_data.get("rss"):
            sources = site_data["rss"]["urls"]
        elif "sitemap" in site_data: # Xử lý cấu trúc sitemap mới
            resolver = DynamicSitemapResolver()
            sitemap_url = resolver.resolve_sitemap_url(site_name, site_data)
            if sitemap_url:
                sources = [sitemap_url]
        elif site_data.get("sitemap_url"): # Cấu trúc cũ
            sources = [site_data["sitemap_url"]]

        if not sources:
            logger.error(f"Không tìm thấy nguồn cho trang {site_name}. Bỏ qua...")
            return

        # Thu thập dữ liệu feed (không crawl chi tiết bài viết)
        logger.info(f"Đang thu thập thông tin feed từ các nguồn: {sources}")
        feed_df = await crawler.prepare_feeder_async(sources)

        # Lọc dữ liệu nếu có điều kiện
        if not feed_df.empty:
            filtered_df = crawler.filter_feeder(feed_df, top_n=top_n, within=within)
        else:
            logger.warning(f"Không có dữ liệu nào được thu thập từ {site_name}")
            return

        # Thiết lập thư mục output
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        output_file = os.path.join(OUTPUT_DIR, f"{site_name}_feed_info.csv")

        # Lưu vào file CSV
        if not filtered_df.empty:
            filtered_df.to_csv(output_file, index=False, encoding="utf-8-sig")
            logger.info(f"Đã lưu {len(filtered_df)} bản ghi vào {output_file}")
        else:
            logger.warning(f"Không có bản ghi nào sau khi lọc cho {site_name}")

    except Exception as e:
        logger.error(f"Lỗi khi xử lý trang '{site_name}': {e}")


async def main():
    """
    Hàm chính để thu thập và lưu thông tin feed cho tất cả các trang được hỗ trợ.
    """
    logger.info("Bắt đầu quá trình xuất thông tin feed cho tất cả các trang...")

    # Tạo danh sách các tác vụ để chạy đồng thời
    tasks = []
    for site_name in SITES_CONFIG.keys():
        task = fetch_feed_info(site_name=site_name, top_n=20, within="1d")
        tasks.append(task)

    # Chạy tất cả các tác vụ đồng thời
    await asyncio.gather(*tasks)

    logger.info("Quá trình xuất thông tin feed đã hoàn tất!")

# --- Cách chạy trong Jupyter Notebook ---
# Dán đoạn code này vào một cell và chạy
await main()

2025-08-31 15:44:17,207 - export_feed_info - INFO - Bắt đầu quá trình xuất thông tin feed cho tất cả các trang...
2025-08-31 15:44:17,208 - export_feed_info - INFO - Đang xử lý trang: cafef
2025-08-31 15:44:17,209 - export_feed_info - INFO - Đang thu thập thông tin feed từ các nguồn: ['https://cafef.vn/latest-news-sitemap.xml']
2025-08-31 15:44:17,210 - export_feed_info - INFO - Đang xử lý trang: cafebiz
2025-08-31 15:44:17,211 - export_feed_info - INFO - Đang thu thập thông tin feed từ các nguồn: ['https://cafebiz.vn/rss/home.rss', 'https://cafebiz.vn/rss/cau-chuyen-kinh-doanh.rss', 'https://cafebiz.vn/rss/vi-mo.rss', 'https://cafebiz.vn/rss/cong-nghe.rss', 'https://cafebiz.vn/rss/xa-hoi.rss']
2025-08-31 15:44:17,211 - export_feed_info - INFO - Đang xử lý trang: vietstock
2025-08-31 15:44:17,212 - export_feed_info - INFO - Đang thu thập thông tin feed từ các nguồn: ['https://vietstock.vn/761/kinh-te/vi-mo.rss', 'https://vietstock.vn/768/kinh-te/kinh-te-dau-tu.rss', 'https://vietstock.

2025-08-31 15:45:20,453 - RSS - INFO - Fetching RSS feed: https://cafebiz.vn/rss/home.rss
2025-08-31 15:45:20,460 - RSS - INFO - Fetching RSS feed: https://cafebiz.vn/rss/vi-mo.rss
2025-08-31 15:45:20,462 - RSS - INFO - Fetching RSS feed: https://cafebiz.vn/rss/cong-nghe.rss
2025-08-31 15:45:20,464 - RSS - INFO - Fetching RSS feed: https://cafebiz.vn/rss/xa-hoi.rss
2025-08-31 15:45:20,466 - RSS - INFO - Fetching RSS feed: https://vietstock.vn/761/kinh-te/vi-mo.rss
2025-08-31 15:45:20,470 - RSS - INFO - Fetching RSS feed: https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss
2025-08-31 15:45:20,473 - RSS - INFO - Fetching RSS feed: https://vietstock.vn/4309/the-gioi/tien-ky-thuat-so.rss
2025-08-31 15:45:20,480 - Sitemap - INFO - Fetching sitemap from https://vneconomy.vn/sitemap/latest-news.xml
2025-08-31 15:45:20,481 - RSS - INFO - Fetching RSS feed: https://vnexpress.net/rss/tin-moi-nhat.rss
2025-08-31 15:45:20,547 - RSS - INFO - Fetching RSS feed: https://tuoitre.vn/rss/tin-moi-

#### Cập nhật tin chi tiết từ nhiều trang

In [6]:
import os
import pandas as pd
import logging
import asyncio
from vnstock_news.async_crawlers.async_batch import AsyncBatchCrawler
from vnstock_news.config.sites import SITES_CONFIG
from vnstock_news.utils.logger import setup_logger
from vnstock_news.config.sitemap_resolver import DynamicSitemapResolver

# Khởi tạo logger
logger = setup_logger("update_news", debug=False)

# Thư mục để lưu file CSV
OUTPUT_DIR = "output/news"

async def fetch_and_save_news(site_name: str, top_n: int = 10, within: str = None):
    """
    Thu thập bài viết chi tiết từ một trang cụ thể và lưu vào file CSV.

    Parameters:
        site_name (str): Tên của trang (từ SITES_CONFIG).
        top_n (int): Số lượng bài viết cần lấy.
        within (str): Khoảng thời gian để lọc bài viết (ví dụ: '1h', '2d').
    """
    try:
        logger.info(f"Đang xử lý trang: {site_name}")
        
        # Khởi tạo AsyncBatchCrawler
        crawler = AsyncBatchCrawler(site_name=site_name)

        # Xác định nguồn (sitemap hoặc RSS)
        site_data = SITES_CONFIG.get(site_name)
        sources = []
        if site_data.get("rss"):
            sources = site_data["rss"]["urls"]
        elif "sitemap" in site_data: # Xử lý cấu trúc sitemap mới
            resolver = DynamicSitemapResolver()
            sitemap_url = resolver.resolve_sitemap_url(site_name, site_data)
            if sitemap_url:
                sources = [sitemap_url]
        elif site_data.get("sitemap_url"): # Cấu trúc cũ
            sources = [site_data["sitemap_url"]]

        if not sources:
            logger.error(f"Không tìm thấy nguồn cho trang {site_name}. Bỏ qua...")
            return

        # Thu thập bài viết chi tiết
        logger.info(f"Đang thu thập bài viết từ các nguồn: {sources}")
        articles_df = await crawler.fetch_articles_async(sources, top_n=top_n, within=within)

        # Thiết lập thư mục output
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        output_file = os.path.join(OUTPUT_DIR, f"{site_name}_news.csv")

        # Lưu vào file CSV
        if not articles_df.empty:
            articles_df.to_csv(output_file, index=False, encoding="utf-8-sig")
            logger.info(f"Đã lưu {len(articles_df)} bài viết vào {output_file}")
        else:
            logger.warning(f"Không có bài viết nào được thu thập từ {site_name}")

    except Exception as e:
        logger.error(f"Lỗi khi xử lý trang '{site_name}': {e}")


async def main():
    """
    Hàm chính để thu thập và lưu tin tức cho tất cả các trang được hỗ trợ.
    """
    logger.info("Bắt đầu quá trình cập nhật tin tức cho tất cả các trang...")

    # Tạo danh sách các tác vụ để chạy đồng thời
    tasks = []
    for site_name in SITES_CONFIG.keys():
        task = fetch_and_save_news(site_name=site_name, top_n=10, within="1d")
        tasks.append(task)

    # Chạy tất cả các tác vụ đồng thời
    await asyncio.gather(*tasks)

    logger.info("Quá trình cập nhật tin tức đã hoàn tất!")

# --- Cách chạy trong Jupyter Notebook ---
# Dán đoạn code này vào một cell và chạy
await main()

2025-08-31 15:45:06,156 - update_news - INFO - Bắt đầu quá trình cập nhật tin tức cho tất cả các trang...
2025-08-31 15:45:06,157 - update_news - INFO - Đang xử lý trang: cafef
2025-08-31 15:45:06,158 - update_news - INFO - Đang thu thập bài viết từ các nguồn: ['https://cafef.vn/latest-news-sitemap.xml']
2025-08-31 15:45:06,158 - update_news - INFO - Đang xử lý trang: cafebiz
2025-08-31 15:45:06,159 - update_news - INFO - Đang thu thập bài viết từ các nguồn: ['https://cafebiz.vn/rss/home.rss', 'https://cafebiz.vn/rss/cau-chuyen-kinh-doanh.rss', 'https://cafebiz.vn/rss/vi-mo.rss', 'https://cafebiz.vn/rss/cong-nghe.rss', 'https://cafebiz.vn/rss/xa-hoi.rss']
2025-08-31 15:45:06,159 - update_news - INFO - Đang xử lý trang: vietstock
2025-08-31 15:45:06,159 - update_news - INFO - Đang thu thập bài viết từ các nguồn: ['https://vietstock.vn/761/kinh-te/vi-mo.rss', 'https://vietstock.vn/768/kinh-te/kinh-te-dau-tu.rss', 'https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss', 'https://viet

## Tuỳ biến chương trình

### Hàm truy xuất nội dung báo cụ thể

#### CafeF

In [ ]:
from vnstock_news.core.crawler import Crawler

# Initialize Crawler for CafeF
cafef_crawler = Crawler("cafef")

# Fetch details of a specific article
sample_url = "https://cafef.vn/dragon-capital-ky-vong-trien-khai-he-thong-krx-va-tien-trinh-nang-hang-thi-truong-se-thu-hut-them-nha-dau-tu-chung-khoan-188241215221837066.chn"
article_details = cafef_crawler.get_article_details(sample_url)
article_details

2025-08-31 14:55:28,953 - Cache - INFO - Initialized memory cache system
2025-08-31 14:55:28,953 - News - INFO - Fetching article HTML: https://cafef.vn/dragon-capital-ky-vong-trien-khai-he-thong-krx-va-tien-trinh-nang-hang-thi-truong-se-thu-hut-them-nha-dau-tu-chung-khoan-188241215221837066.chn


{'title': 'Dragon Capital: Kỳ vọng triển khai hệ thống KRX và tiến trình nâng hạng thị trường sẽ thu hút thêm nhà đầu tư chứng khoán',
 'short_description': 'So với giai đoạn căng thẳng thương mại năm 2018, khi rủi ro gia tăng bởi định giá cao và các đợt niêm yết mới, Dragon Capital cho rằng thị trường hiện tại đã phản ánh phần lớn những bất ổn hiện hữu.',
 'publish_time': '16-12-2024 - 00:04 AM',
 'author': 'Mai Chi',
 'markdown_content': '[TIN MỚI](javascript:; "TIN MỚI")\n\nTrong báo cáo mới đây, ông Nguyễn Quang Hưng, CFA Chuyên gia Kinh tế cấp cao\nDragon Capital cho biết chỉ số kinh tế của Việt Nam trong tháng 11 phản\nánh bức tranh đa chiều. Hoạt động xuất khẩu tăng 8,2% so với cùng\nkỳ năm trước, đạt 33,7 tỷ USD; hoạt động nhập khẩu tăng 13,6% đạt\n32,6 tỷ USD. Thặng dư thương mại khoảng 1,1 tỷ USD. Ngành sản xuất\ncông nghiệp cũng ghi nhận sự tăng trưởng, chỉ số sản xuất công\nnghiệp tăng 2,3% so với tháng trước và 8,

#### VnExpress

In [ ]:
from vnstock_news.core.crawler import Crawler

# Initialize Crawler for VNExpress
vnexpress_crawler = Crawler("vnexpress")

# Fetch details of a specific article
sample_url = "https://vnexpress.net/bi-thu-tinh-uy-vinh-phuc-chon-can-bo-tam-huyet-uy-tin-sau-tinh-gon-4828307.html"
article_details = vnexpress_crawler.get_article_details(sample_url)
article_details

2025-08-31 14:55:36,431 - News - INFO - Fetching article HTML: https://vnexpress.net/bi-thu-tinh-uy-vinh-phuc-chon-can-bo-tam-huyet-uy-tin-sau-tinh-gon-4828307.html


{'title': "'Tránh chọn Lý Thông, bỏ qua Thạch Sanh khi sắp xếp bộ máy'",
 'short_description': 'Để tránh bỏ qua người có năng lực khi sắp xếp bộ máy, Bí thư Tỉnh ủy Vĩnh Phúc xác định cần bám tiêu chuẩn chức danh kết hợp đánh giá uy tín, trách nhiệm, tâm huyết của cán bộ.',
 'publish_time': 'Thứ hai, 16/12/2024, 11:51 (GMT+7)',
 'author': 'Chọn sai người gây tổn hại ít nhất 5 năm',
 'markdown_content': 'Hôm 5/12, ông Lê Minh Hưng, Trưởng ban Tổ chức Trung ương kiêm Phó trưởng ban\nthường trực Ban Chỉ đạo Trung ương về tổng kết Nghị quyết 18, đã ký công văn\ntriển khai nhiệm vụ tổng kết Nghị quyết để các cấp ủy, tổ chức đảng, cơ quan,\nđơn vị trực thuộc Trung ương nghiên cứu đề xuất sắp xếp, hoàn thiện tổ chức bộ\nmáy của hệ thống chính trị tinh gọn. Cùng ngày, Phó thủ tướng Thường trực\nNguyễn Hòa Bình cũng công bố kế hoạch sắp xếp tổ chức bộ máy của Chính phủ,\ntrong đó định hướng sắp xếp 14 bộ, cơ quan ngang bộ.\n\nTrong 10 ngày sau đó, cùng với các bộ ngành Trung ương, hơn 10 tỉnh, 

In [ ]:
from vnstock_news.core.rss import RSS

rss = RSS(site_name="vnexpress", description_format='text')
rss.fetch()

2025-08-31 14:55:37,124 - RSS - INFO - Fetching RSS feed: https://vnexpress.net/rss/tin-moi-nhat.rss


[{'link': 'https://vnexpress.net/di-chuyen-mot-que-diem-de-sua-phep-tinh-sai-4931779.html',
  'title': 'Di chuyển một que diêm để sửa phép tính sai',
  'description': "Nghe đơn giản, nhưng tin mình đi, hơn 90% người thử đều phải vò đầu bứt tai, thậm chí 'cà khịa chính mình' vì… nghĩ mãi không ra.",
  'publish_time': 'Sun, 31 Aug 2025 14:00:00 +0700'},
 {'link': 'https://vnexpress.net/nt-logistics-ra-mat-dich-vu-dinh-tuyen-giup-cat-giam-phi-van-tai-4932052.html',
  'title': 'NT Logistics ra mắt dịch vụ định tuyến giúp cắt giảm phí vận tải',
  'description': 'Dịch vụ RaaS+ của NT Logistics giúp tối ưu đội xe và nâng cao hiệu quả giao hàng nhờ kết hợp công nghệ định tuyến với chuyên gia.',
  'publish_time': 'Sun, 31 Aug 2025 14:00:00 +0700'},
 {'link': 'https://vnexpress.net/le-hoang-phuong-dien-vay-20-kg-lam-vedette-4933708.html',
  'title': 'Lê Hoàng Phương diện váy 20 kg làm vedette',
  'description': 'Hoa hậu Lê Hoàng Phương diễn đầm dạ hội "Mảnh sáng lấp lánh" với trọng lượng lớn của

#### Tuổi Trẻ

In [ ]:
from vnstock_news.core.crawler import Crawler

# Initialize Crawler for tuoitre
tuoitre_crawler = Crawler("tuoitre")

# Fetch details of a specific article
url = "https://tuoitre.vn/tong-bi-thu-to-lam-tinh-gon-bo-may-la-van-de-kho-phai-khach-quan-than-trong-dan-chu-20241216161830849.htm"
article_details = tuoitre_crawler.get_article_details(url)
article_details

2025-08-31 14:55:37,228 - News - INFO - Fetching article HTML: https://tuoitre.vn/tong-bi-thu-to-lam-tinh-gon-bo-may-la-van-de-kho-phai-khach-quan-than-trong-dan-chu-20241216161830849.htm


{'title': 'Tổng Bí thư Tô Lâm: Tinh gọn bộ máy là vấn đề khó, phải khách quan, thận trọng, dân chủ',
 'short_description': 'Tổng Bí thư Tô Lâm nêu rõ phải làm tốt công tác tham mưu, sắp xếp tinh gọn tổ chức bộ máy của hệ thống chính trị gắn với cơ cấu lại đội ngũ cán bộ.',
 'publish_time': '16/12/2024 17:01 GMT+7',
 'author': None,
 'markdown_content': '![Tổng Bí thư Tô Lâm: Tinh gọn bộ máy là vấn đề khó, phải khách quan, thận\ntrọng, dân chủ - Ảnh\n1.](https://cdn2.tuoitre.vn/thumb_w/730/471584752817336320/2024/12/16/tbt3-17343397917111271024338.png)\n\nTổng Bí thư Tô Lâm - Ảnh: TTXVN\n\nNgày 16-12, Ban Tổ chức Trung ương đã tổ chức Hội nghị toàn quốc tổng kết công\ntác tổ chức [xây dựng Đảng](https://tuoitre.vn/xay-dung-dang.html "xây dựng\nĐảng") năm 2024, triển khai nhiệm vụ năm 2025 và trực tuyến tới các điểm cầu\ntrên cả nước.\n\n## Đội ngũ cán bộ phải tâm huyết vì nước, vì dân, vì sự nghiệp của Đảng\n\nPhát biểu chỉ đạo tại đây, [Tổng Bí thư Tô Lâm](/tong-bi-thu-to-lam.html "Tổn

#### VNEconomy

In [ ]:
from vnstock_news.core.crawler import Crawler

# Initialize Crawler for VNExpress
vneconomy_crawler = Crawler("vneconomy")

# Fetch details of a specific article
sample_url = "https://vneconomy.vn/thanh-khoan-dong-bang-phien-dau-tuan-vn-index-tiep-tuc-thu-thach-moc-1260-diem.htm"
article_details = vneconomy_crawler.get_article_details(sample_url)
article_details

2025-08-31 14:55:37,348 - News - INFO - Fetching article HTML: https://vneconomy.vn/thanh-khoan-dong-bang-phien-dau-tuan-vn-index-tiep-tuc-thu-thach-moc-1260-diem.htm


{'title': 'Thanh khoản “đóng băng” phiên đầu tuần, VN-Index tiếp tục thử thách mốc 1260 điểm',
 'short_description': 'Nhà đầu tư giảm mạnh cường độ trong phiên giao dịch sáng nay khi chờ đợi các sự kiện quan trọng xuất hiện. Kỳ họp của FED, thời điểm đáo hạn phái sinh và đợt tái cơ cấu ETF ngoại sẽ hội tụ tuần này. Thanh khoản sụt giảm mạnh trong thời điểm như vậy là bình thường...',
 'publish_time': '16/12/2024, 12:15',
 'author': 'Kim  Phong',
 'markdown_content': '**Nhà đầu tư giảm mạnh cường độ trong phiên giao dịch sáng nay khi chờ đợi các\nsự kiện quan trọng xuất hiện. Kỳ họp của FED, thời điểm đáo hạn phái sinh và\nđợt tái cơ cấu ETF ngoại sẽ hội tụ tuần này. Thanh khoản sụt giảm mạnh trong\nthời điểm như vậy là bình thường...**\n\n![Vẫn có một số cổ phiếu thu hút dòng tiền tốt hơn phần còn lại và giá tăng\ntốt.](https://media.vneconomy.vn/images/upload/2024/12/16/bong-bong.png?w=900)\nVẫn có một số cổ phiếu thu hút dòng tiền tốt hơn phần còn lại và giá tăng tốt.\n\nNhà đầu tư g

#### Pháp Luật Online

In [ ]:
from vnstock_news.core.crawler import Crawler

# Initialize Crawler for VNExpress
plo_crawler = Crawler("plo")

# Fetch details of a specific article
sample_url = "https://plo.vn/can-bo-thue-nhan-hoi-lo-bao-ke-cho-duong-day-mua-ban-hoa-don-bi-de-nghi-den-20-nam-tu-post825196.html"
article_details = plo_crawler.get_article_details(sample_url)
article_details

2025-08-31 14:55:37,493 - News - INFO - Fetching article HTML: https://plo.vn/can-bo-thue-nhan-hoi-lo-bao-ke-cho-duong-day-mua-ban-hoa-don-bi-de-nghi-den-20-nam-tu-post825196.html


{'title': 'Cán bộ thuế nhận hối lộ, bảo kê cho đường dây mua bán hóa đơn bị đề nghị đến 20 năm tù',
 'short_description': '(PLO)- Theo VKS, để che giấu hành vi mua bán trái phép hóa đơn, các bị cáo đã móc nối và thỏa thuận đưa hối lộ cho các cán bộ thuế tại nơi có công ty "ma" hoạt động.',
 'publish_time': '31/08/2025 09:00',
 'author': 'SONG MAI',
 'markdown_content': 'Ngày 16-12, đại diện VKSND TP.HCM nêu quan điểm luận tội và đề nghị mức án đối\nvới 55 bị cáo trong vụ _cán bộ[thuế](https://plo.vn/kinh-te/ "thuế") nhận hối\nlộ, bảo kê cho đường dây mua bán trái phép hóa đơn_.\n\nĐại diện VKS đề nghị mức án đối với bị cáo Lê Thành Nhân (công chức Chi cục\nThuế Khu vực Quận 12 - Hóc Môn) 18-20 năm tù, Trần Quốc Duy (công chức Chi cục\nThuế Khu vực quận 7 - huyện Nhà Bè) 15-16 năm tù; Vương Quốc Hùng (cựu Phó đội\ntrưởng Đội kiểm tra nội bộ - Chi cục Thuế TP Thủ Dầu Một, Bình Dương) 7-8 năm\ntù cùng về tội nhận hối lộ.\n\nBị cáo Bùi Thanh Liêm (cựu Phó đội trưởng Đội kiểm tra thuế số 3,

#### Kinh tế SG Online

In [ ]:
from vnstock_news.core.crawler import Crawler

# Initialize Crawler for VNExpress
crawler = Crawler("ktsg")

# Fetch details of a specific article
url = "https://thesaigontimes.vn/ong-donald-trump-thang-cu-thao-tung-tien-te-va-ty-gia/"
article_details = crawler.get_article_details(url)
article_details

2025-08-31 14:55:52,858 - DynamicSitemapResolver - ERROR - Error resolving incremental sitemap from https://thesaigontimes.vn/sitemap_index.xml: HTTPSConnectionPool(host='thesaigontimes.vn', port=443): Read timed out. (read timeout=15)
2025-08-31 14:55:52,859 - News - INFO - Fetching article HTML: https://thesaigontimes.vn/ong-donald-trump-thang-cu-thao-tung-tien-te-va-ty-gia/


{'title': 'Ông Donald Trump thắng cử, thao túng tiền tệ, và tỷ giá',
 'short_description': '(KTSG) - Trong cuộc bầu cử tổng thống Mỹ vừa qua, ông Donald Trump không chỉ giành chiến thắng áp đảo mà đảng Cộng hòa cũng chiếm đa số trong cả hai viện Quốc hội. Kết quả này sẽ có những tác động sâu rộng không chỉ đối với Mỹ mà cả thế giới. Trong đó, có câu chuyện tỷ giá hối đoái đối với các nước có thặng dư thương mại lớn với Mỹ, mà Việt Nam là một trường hợp khá đặc biệt.',
 'publish_time': '10:00 21/11/2024',
 'author': 'TS. Võ Đình Trí',
 'markdown_content': '(KTSG) - Trong cuộc bầu cử tổng thống Mỹ vừa qua, ông Donald Trump không chỉ\ngiành chiến thắng áp đảo mà đảng Cộng hòa cũng chiếm đa số trong cả hai viện\nQuốc hội. Kết quả này sẽ có những tác động sâu rộng không chỉ đối với Mỹ mà cả\nthế giới. Trong đó, có câu chuyện tỷ giá hối đoái đối với các nước có thặng dư\nthương mại lớn với Mỹ, mà Việt Nam là một trường hợp khá đặc biệt.\n\n  * [Ngành chip Trung Quốc chuẩn bị cho thách thức T

#### Nhịp cầu đầu tư

In [ ]:
from vnstock_news.core.crawler import Crawler

# Initialize Crawler for VNExpress
crawler = Crawler("ncdt")

# Fetch details of a specific article
url = "https://nhipcaudautu.vn/kinh-doanh/lua-dao-quoc-te-bua-vay-3361191/"
article_details = crawler.get_article_details(url)
article_details

2025-08-31 14:55:55,521 - News - INFO - Fetching article HTML: https://nhipcaudautu.vn/kinh-doanh/lua-dao-quoc-te-bua-vay-3361191/


{'title': 'Lừa đảo quốc tế bủa vây',
 'short_description': 'Tưởng rằng đang kiếm được tiền trong các hợp đồng thương mại “béo bở”, nhưng hóa ra doanh nghiệp Việt đã bị lừa.',
 'publish_time': 'Thứ Sáu |13/12/2024 14:00',
 'author': 'Trung Nam',
 'markdown_content': '  \n\nTưởng rằng đang kiếm được tiền trong các hợp đồng thương mại “béo bở”, nhưng\nhóa ra doanh nghiệp Việt đã bị lừa.\n\n  \n\nĐầu tháng 10/2024, Hiệp hội Hồ tiêu và Cây gia vị Việt Nam (VPSA) phát đi cảnh\nbáo về trường hợp lừa đảo của một khách hàng tại Malaysia. Dù nội tình vụ việc\nchưa được tiết lộ chi tiết song được biết, lô hàng nông sản bị đối tác lừa đảo\nlấy ra khỏi cảng trong khi chủ hàng vẫn giữ bộ vận đơn đường biển gốc, nghĩa\nlà thủ tục chưa hoàn tất. Đối tượng này đã lừa được 3 công ty Việt Nam.\n\n**Vạn trạng thủ đoạn**\n\nTrước đó, năm 2022, 5 doanh nghiệp thành viên thuộc Hiệp hội Điều Việt Nam\n(Vinacas) cũng mất quyền kiểm soát với 36 container hàng xuất sang Ý, tương\nđương 36 bộ chứng từ gốc đi kè

#### Cafebiz

In [ ]:
from vnstock_news.core.crawler import Crawler

# Initialize Crawler for VNExpress
crawler = Crawler("cafebiz")

# Fetch details of a specific article
url = "https://cafebiz.vn/khoi-to-chu-tich-phuong-nenh-o-bac-giang-ve-hanh-vi-nhan-hoi-lo-176241216211627623.chn"
article_details = crawler.get_article_details(url)
article_details

2025-08-31 14:55:55,766 - News - INFO - Fetching article HTML: https://cafebiz.vn/khoi-to-chu-tich-phuong-nenh-o-bac-giang-ve-hanh-vi-nhan-hoi-lo-176241216211627623.chn


{'title': 'Khởi tố Chủ tịch phường Nếnh ở Bắc Giang về hành vi nhận hối lộ',
 'short_description': 'Ngày 15/12, Cơ quan Cảnh sát điều tra Công an tỉnh Bắc Giang khởi tố Đỗ Văn Luận (SN 1982), Chủ tịch UBND phường Nếnh, thị xã Việt Yên (Bắc Giang) về hành vi nhận hối lộ.',
 'publish_time': '16/12/2024 21:15 PM',
 'author': 'Văn Giang',
 'markdown_content': 'Trước đó, vào lúc 17 giờ 55 phút ngày 6/12/2024, Cơ quan Cảnh sát điều tra\nCông an tỉnh Bắc Giang đã bắt quả tang Đỗ Văn Luận có hành vi nhận hối lộ số\ntiền 60 triệu đồng của ông Nguyễn Văn Sen (SN 1972), trú tại phường Dĩnh Kế\n(TP Bắc Giang) tại phòng làm việc của Đỗ Văn Luận.\n\n![Khởi tố Chủ tịch phường Nếnh ở Bắc Giang về hành vi nhận hối lộ- Ảnh\n1.](https://cafebiz.cafebizcdn.vn/162123310254002176/2024/12/16/i-t-ng-v-n-\nlu-n-2-1734358522194-1734358522384478799274.jpg)\n\nĐỗ Văn Luận và tang vật bị thu giữ\n\nĐỗ Văn Luận và ông Nguyễn Văn Sen đều khai nhận đây là số tiền ông Sen đưa cho\nLuận để thực hiện việc quyết toán tiề

#### Diễn đàn doanh nghiệp

In [ ]:
from vnstock_news.core.crawler import Crawler

# Initialize Crawler for VNExpress
crawler = Crawler("dddn")

# Fetch details of a specific article
url = "https://diendandoanhnghiep.vn/go-kho-cho-kinh-te-tuan-hoan-trong-khu-cong-nghiep-10147366.html"
article_details = crawler.get_article_details(url)
article_details

2025-08-31 14:55:56,083 - News - INFO - Fetching article HTML: https://diendandoanhnghiep.vn/go-kho-cho-kinh-te-tuan-hoan-trong-khu-cong-nghiep-10147366.html


{'title': 'Gỡ khó cho kinh tế tuần hoàn trong khu công nghiệp',
 'short_description': 'Nhiều doanh nghiệp Việt Nam đang nỗ lực chuyển đổi mô hình kinh tế tuần hoàn, nhất là tại các khu công nghiệp nhưng quá trình này đang có rào cản.',
 'publish_time': '16/12/2024 02:51',
 'author': 'Hạnh Lê',
 'markdown_content': 'Doanh nghiệp\n\n# Gỡ khó cho kinh tế tuần hoàn trong khu công nghiệp\n\nHạnh Lê • 16/12/2024 02:51\n\nNhiều doanh nghiệp Việt Nam đang nỗ lực chuyển đổi mô hình kinh tế tuần hoàn,\nnhất là tại các khu công nghiệp nhưng quá trình này đang có rào cản.\n\nTrong quá trình thúc đẩy nhân rộng mô hình kinh tế tuần hoàn trong các ngành\nnghề, lĩnh vực của nền kinh tế hướng đến mục tiêu chung là giảm phát thải,\nthực hiện Netzero vào năm 2030, các chuyên gia nhấn mạnh đến vai trò quan\ntrọng của mô hình khu công nghiệp sinh thái (EIP). Ông Jerome Stucki - Trưởng\nbộ phận Kinh tế tuần hoàn và tài nguyên của Tổ chức Phát triển công nghiệp\nLiên Hợp Quốc (UNIDO) nhấn mạnh, khu vực này c

#### Báo đầu tư

In [ ]:
from vnstock_news.core.crawler import Crawler

# Initialize Crawler for VNExpress
crawler = Crawler("baodautu")

# Fetch details of a specific article
url = "https://baodautu.vn/thuc-cong-nghe-chien-luoc-de-tang-toc-phat-trien-kinh-te-d371629.html"
article_details = crawler.get_article_details(url)
article_details

2025-08-31 14:55:57,310 - News - INFO - Fetching article HTML: https://baodautu.vn/thuc-cong-nghe-chien-luoc-de-tang-toc-phat-trien-kinh-te-d371629.html


{'title': None,
 'short_description': 'Để nền kinh tế Việt Nam có thể tăng tốc phát triển, một trong những điểm mấu chốt là thúc đẩy sự hình thành và phát triển của các ngành công nghệ chiến lược, bao gồm bán dẫn, trí tuệ nhân tạo (AI)…',
 'publish_time': '- 31/08/2025 08:43',
 'author': 'Nguyên Đức',
 'markdown_content': '![](https://media.Baodautu.vn/Images/phuongthanh02/2025/08/26/a t4.jpg)  \n---  \nViệc hình thành và phát triển được các ngành công nghiệp chiến lược có thể\ngiúp Việt Nam bắt kịp, tiến cùng và vượt lên trong một số lĩnh vực trọng yếu.\nẢnh: Đức Thanh  \n  \n**Bước khởi đầu cho các ngành công nghệ chiến lược**\n\nĐồng loạt 3 mạng lưới đổi mới sáng tạo và chuyên gia ngành công nghệ chiến\nlược, bao gồm Lượng tử (VNQuantum), An ninh mạng (ViSecurity) và Hàng không,\nvũ trụ, thiết bị bay không người lái (UAV Việt Nam), vừa được Trung tâm Đổi\nmới sáng tạo Quốc gia (Bộ [Tài chính](http://Baodautu.vn/tai-chinh-bao-\nhiem-d6/)) chính thức công bố thành lập.\n\nTrong đó, Mạ

#### Vietstock

In [ ]:
from vnstock_news.core.crawler import Crawler

# Initialize Crawler for VNExpress
crawler = Crawler("vietstock")

# Fetch details of a specific article
url = "https://vietstock.vn/2024/12/nhip-dap-thi-truong-1812-sac-xanh-bao-phu-hau-het-cac-nhom-nganh-1636-1253348.htm"
article_details = crawler.get_article_details(url)
article_details

2025-08-31 14:55:58,135 - News - INFO - Fetching article HTML: https://vietstock.vn/2024/12/nhip-dap-thi-truong-1812-sac-xanh-bao-phu-hau-het-cac-nhom-nganh-1636-1253348.htm


{'title': 'Nhịp đập Thị trường 18/12: Nhóm dầu khí dậy sóng, VN-Index lấy lại mốc 1,265 điểm',
 'short_description': 'Kết thúc phiên giao dịch,VN-Indextăng 4.28 điểm (0.34%), lên mức 1,266 điểm;HNX-Indextăng 0.54 điểm (0.24%), lên mức 227.43 điểm. Độ rộng toàn thị trường nghiêng về sắc xanh với bên mua có 438 mã tăng và bên bán có 267 mã giảm. Sắc xanh có phần áp đảo trong rổVN30với 18 mã tăng, 7 mã giảm và 5 mã tham chiếu.',
 'publish_time': '18/12/2024 15:22',
 'author': 'Lý Hỏa',
 'markdown_content': 'Nhịp đập Thị trường 18/12: Nhóm dầu khí dậy sóng, VN-Index lấy lại mốc 1,265\nđiểm\n\nKết thúc phiên giao dịch, VN-Index tăng 4.28 điểm (0.34%), lên mức 1,266 điểm;\nHNX-Index tăng 0.54 điểm (0.24%), lên mức 227.43 điểm. Độ rộng toàn thị trường\nnghiêng về sắc xanh với bên mua có 438 mã tăng và bên bán có 267 mã giảm. Sắc\nxanh có phần áp đảo trong rổ [VN30](https://finance.vietstock.vn/ket-qua-giao-\ndich/vietnam.aspx?languageid=1) với 18 mã tăng, 7 mã giảm và 5 mã tham chiếu.\n\nThan

### Truy xuất danh sách bài viết

#### Nguồn sitemap

In [7]:
import pandas as pd
from vnstock_news.core.crawler import Crawler

# Cách 1: Sử dụng Crawler trực tiếp
crawler = Crawler("cafef", use_predefined_config=True)
articles_dict = crawler.get_articles(sitemap_url="https://cafef.vn/latest-news-sitemap.xml", limit=5)
cafef_sitemap_df = pd.DataFrame(articles_dict)
cafef_sitemap_df

2025-08-31 16:05:10,257 - Crawler - INFO - Fetching articles via Sitemap (fallback): https://cafef.vn/latest-news-sitemap.xml
2025-08-31 16:05:10,258 - Sitemap - INFO - Fetching sitemap from https://cafef.vn/latest-news-sitemap.xml


,url,lastmod
0,https://cafef.vn/chi-vi-ten-xuat-hien-trong-ti...,2025-08-31 15:58:06+07:00
1,https://cafef.vn/nhan-vien-dau-tien-cua-viette...,2025-08-31 15:45:04+07:00
2,https://cafef.vn/tong-thong-nga-bat-dau-chuyen...,2025-08-31 15:38:55+07:00
3,https://cafef.vn/4-con-giap-sap-don-loat-tin-v...,2025-08-31 14:35:16+07:00
4,https://cafef.vn/giao-su-dai-hoc-thanh-hoa-tha...,2025-08-31 14:02:14+07:00


#### Nguồn RSS

In [10]:
import pandas as pd
from vnstock_news.core.crawler import Crawler

# Cách 1: Sử dụng Crawler trực tiếp
crawler = Crawler("vnexpress", use_predefined_config=True)
articles_dict = crawler.get_articles(sitemap_url="https://vnexpress.net/rss/tin-moi-nhat.rss", limit=5)
vne_sitemap_df = pd.DataFrame(articles_dict)
vne_sitemap_df

2025-08-31 16:09:11,736 - Crawler - INFO - Fetching articles via RSS (preferred)
2025-08-31 16:09:11,738 - RSS - INFO - Fetching RSS feed: https://vnexpress.net/rss/tin-moi-nhat.rss


,title,description,publish_time,url
0,Lãnh đạo quốc hội Trung Quốc đến Việt Nam,Lãnh đạo Ủy ban Thường vụ Đại hội Đại biểu Nhâ...,"Sun, 31 Aug 2025 15:06:39 +0700",https://vnexpress.net/lanh-dao-quoc-hoi-trung-...
1,Những công nghệ giúp tàu Nhật Bản đến đúng giờ,Các nhà chức trách Nhật Bản sử dụng phối hợp n...,"Sun, 31 Aug 2025 15:00:00 +0700",https://vnexpress.net/nhung-cong-nghe-giup-tau...
2,Người đàn ông luôn hướng về gia đình tìm bạn đời,"Anh sống khá nội tâm và tình cảm, ít nói nhưng...","Sun, 31 Aug 2025 15:00:00 +0700",https://vnexpress.net/nguoi-dan-ong-luon-huong...
3,LG CNS Việt Nam tổ chức chương trình Tech Insi...,LG CNS Vietnam Build Center và Học viện Công n...,"Sun, 31 Aug 2025 15:00:00 +0700",https://vnexpress.net/lg-cns-viet-nam-to-chuc-...
4,Tôn Nam Kim trưng bày sản phẩm đặc biệt tại tr...,Nam Kim đưa tôn Zinmag có độ bền vượt trội và ...,"Sun, 31 Aug 2025 15:00:00 +0700",https://vnexpress.net/ton-nam-kim-trung-bay-sa...


### Lấy toàn bộ nội dung bài viết

#### Nguồn sitemap

In [11]:
from vnstock_news.core.batch import BatchCrawler

custom_config = {
    "site_name": "cafef",
    "sitemap_url": "https://cafef.vn/latest-news-sitemap.xml"
}
batch_crawler = BatchCrawler(site_name="cafef", custom_config=custom_config)
articles_df = batch_crawler.fetch_articles("https://cafef.vn/latest-news-sitemap.xml", limit=10, top_n_per_feed=5)
articles_df

2025-08-31 16:13:52,480 - Crawler - INFO - Fetching articles via Sitemap (fallback): https://cafef.vn/latest-news-sitemap.xml
2025-08-31 16:13:52,481 - Sitemap - INFO - Fetching sitemap from https://cafef.vn/latest-news-sitemap.xml
Fetching article details: 100%|██████████| 10/10 [00:11<00:00,  1.19s/it]


,title,short_description,publish_time,author,markdown_content,url
0,"Sớm vượt quy hoạch, Sân bay Cam Ranh được đề x...",Theo quy hoạch được duyệt tại Quyết định số 10...,31-08-2025 - 16:10 PM,Theo Phan Trang,"[TIN MỚI](javascript:; ""TIN MỚI"")\n\n![Sớm vượ...",https://cafef.vn/som-vuot-quy-hoach-san-bay-ca...
1,Công an Hà Nội cảnh báo khẩn liên quan đến diễ...,"Cảnh báo lừa đảo bán giấy mời xem diễu binh, d...",31-08-2025 - 16:09 PM,MT,"[TIN MỚI](javascript:; ""TIN MỚI"")\n\n![Công an...",https://cafef.vn/cong-an-ha-noi-canh-bao-khan-...
2,"Nam sinh Hải Phòng phá két, lấy vàng đi bán để...",Công an phường An Biên (TP Hải Phòng) vừa ngăn...,31-08-2025 - 16:08 PM,Theo Nguyễn Hoàn,"[TIN MỚI](javascript:; ""TIN MỚI"")\n\nCông an p...",https://cafef.vn/nam-sinh-hai-phong-pha-ket-la...
3,Harper Beckham: Công chúa nhà Becks thành thạo...,"Harper Seven Beckham, cô con gái út của David ...",31-08-2025 - 16:02 PM,"Theo Tú Tú,","[TIN MỚI](javascript:; ""TIN MỚI"")\n\nHarper Be...",https://cafef.vn/harper-beckham-cong-chua-nha-...
4,Hàn Quốc ban bố tình trạng thảm họa quốc gia d...,Tổng thống Hàn Quốc Lee Jae Myung đã ban bố tì...,31-08-2025 - 16:01 PM,Theo Mạnh Dương,"[TIN MỚI](javascript:; ""TIN MỚI"")\n\n![Tổng th...",https://cafef.vn/han-quoc-ban-bo-tinh-trang-th...
5,Chỉ vì tên xuất hiện trong tin nhắn yêu cầu ch...,Công an Hà Nội giải cứu thành công cô gái trẻ ...,31-08-2025 - 15:57 PM,MT,"[TIN MỚI](javascript:; ""TIN MỚI"")\n\n![Chỉ vì ...",https://cafef.vn/chi-vi-ten-xuat-hien-trong-ti...
6,Nhân viên đầu tiên của Viettel được phong tặng...,"Gia nhập Viettel năm 2008, đồng chí Lê Thị Hằn...",31-08-2025 - 15:48 PM,Đinh Anh,"[TIN MỚI](javascript:; ""TIN MỚI"")\n\nTheo Quyế...",https://cafef.vn/nhan-vien-dau-tien-cua-viette...
7,Tổng thống Nga bắt đầu chuyến thăm Trung Quốc,Tổng thống Nga Vladimir Putin đã đến Thiên Tân...,31-08-2025 - 15:38 PM,Theo Minh Hạnh,"[TIN MỚI](javascript:; ""TIN MỚI"")\n\n![Tổng th...",https://cafef.vn/tong-thong-nga-bat-dau-chuyen...
8,"4 con giáp sắp đón loạt tin vui tới tấp, ra cử...",Tài lộc của con giáp này có dấu hiệu leo thang...,31-08-2025 - 15:38 PM,Thùy Linh,"[TIN MỚI](javascript:; ""TIN MỚI"")\n\n### Tuổi ...",https://cafef.vn/4-con-giap-sap-don-loat-tin-v...
9,Giáo sư Đại học Thanh Hoa tham gia dẫn dắt phá...,"Ông Đường Kiệt, giáo sư hàng đầu tại Đại học T...",31-08-2025 - 15:31 PM,Theo Anh Nhật,"[TIN MỚI](javascript:; ""TIN MỚI"")\n\nTheo thôn...",https://cafef.vn/giao-su-dai-hoc-thanh-hoa-tha...


#### Nguồn RSS

In [ ]:
from vnstock_news.core.batch import BatchCrawler

# Fetch articles from a predefined sitemap
crawler = BatchCrawler("vnexpress", debug=False)
articles_df = crawler.fetch_articles(["https://vnexpress.net/rss/tin-moi-nhat.rss", "https://vnexpress.net/rss/thoi-su.rss"], limit=10, top_n_per_feed=5)
articles_df

2025-08-31 16:06:48,140 - Crawler - INFO - Fetching articles via RSS (preferred)
2025-08-31 16:06:48,142 - RSS - INFO - Fetching RSS feed: https://vnexpress.net/rss/tin-moi-nhat.rss
Fetching article details: 100%|██████████| 10/10 [00:11<00:00,  1.12s/it]


,title,short_description,publish_time,author,markdown_content,url
0,Khách Tây lạ lẫm giữa 'rừng' cờ đỏ sao vàng ở ...,"Những con phố quen thuộc được ""nhuộm đỏ"" bởi c...","Chủ nhật, 31/8/2025, 14:48 (GMT+7)",Mai Phương,![Khách Tây lạ lẫm giữa 'rừng' cờ đỏ sao vàng ...,https://vnexpress.net/khach-tay-la-lam-giua-ru...
1,"Sinner vất vả, đứt dây giày để vào vòng 1/8 Mỹ...",MỹTay vợt số một thế giới Jannik Sinner đã phả...,"Chủ nhật, 31/8/2025, 14:43 (GMT+7)",Hoàng Thông,Sau hai trận dễ dàng trước Vit Kopriva (xếp th...,https://vnexpress.net/sinner-vat-va-dut-day-gi...
2,Đề xuất thời gian cấp phép xây dựng tối đa 7 ngày,Bộ Xây dựng đề xuất rút ngắn thời gian cấp giấ...,"Chủ nhật, 31/8/2025, 14:05 (GMT+7)",Phương Uyên,Tại dự thảo Luật Xây dựng sửa đổi đang lấy ý k...,https://vnexpress.net/de-xuat-thoi-gian-cap-ph...
3,Di chuyển một que diêm để sửa phép tính sai,"Nghe đơn giản, nhưng tin mình đi, hơn 90% ngườ...","Chủ nhật, 31/8/2025, 14:00 (GMT+7)",>> Xem đáp án,Hãy cùng đón nhận thử thách tư duy đỉnh cao vớ...,https://vnexpress.net/di-chuyen-mot-que-diem-d...
4,NT Logistics ra mắt dịch vụ định tuyến giúp cắ...,MỹDịch vụ RaaS+ của NT Logistics giúp tối ưu đ...,"Chủ nhật, 31/8/2025, 14:00 (GMT+7)",Hải My,NT Logistics vừa giới thiệu dịch vụ Routing-as...,https://vnexpress.net/nt-logistics-ra-mat-dich...
5,Lê Hoàng Phương diện váy 20 kg làm vedette,TP HCMHoa hậu Lê Hoàng Phương diễn đầm dạ hội ...,"Chủ nhật, 31/8/2025, 13:35 (GMT+7)",Tân Cao,![](https://i1-giaitri.vnecdn.net/2025/08/31/B...,https://vnexpress.net/le-hoang-phuong-dien-vay...
6,Tổng Bí thư dâng hương tại nơi Chủ tịch Hồ Chí...,"Hà NộiSáng 31/8, Tổng Bí thư Tô Lâm đến dâng h...","Chủ nhật, 31/8/2025, 13:30 (GMT+7)",Vũ Tuân,Cùng tham gia dâng hương có Bí thư Thành ủy Hà...,https://vnexpress.net/tong-bi-thu-dang-huong-t...
7,Vingroup lãi hơn 2.000 tỷ đồng từ thoái vốn cô...,"Nửa đầu năm, Vingroup lãi hơn 2.000 tỷ đồng từ...","Chủ nhật, 31/8/2025, 13:30 (GMT+7)",Trọng Hiếu,Thông tin trên được Vingroup nêu trong báo cáo...,https://vnexpress.net/vingroup-lai-hon-2-000-t...
8,Ôtô lao vào cửa hàng điện thoại,Đồng NaiNữ tài xế lái ôtô 4 chỗ lao lên vỉa hè...,"Chủ nhật, 31/8/2025, 13:29 (GMT+7)",Phước Tuấn,"Khoảng 9h, nữ tài xế khoảng 45 tuổi, lái ôtô c...",https://vnexpress.net/oto-lao-vao-cua-hang-die...
9,Bị nhầm là 'trà xanh' khi đi cùng bạn trai hơn...,"Anh gần 50 tuổi nhưng vẫn phong độ: cao ráo, ă...","Chủ nhật, 31/8/2025, 13:23 (GMT+7)",Hồng Lý,"Tôi 29 tuổi, là nhân viên marketing. Chưa bao ...",https://vnexpress.net/yeu-nguoi-hon-nhieu-tuoi...


### Tự định nghĩa cấu hình trang

In [39]:
custom_site_config = {
    "site_name": "dantri", 
    "config": {
        "title_selector": {"tag": "h1", "class": "title-page"},
        "content_selector": {"tag": "div", "class": "singular-content"},
        "short_desc_selector": {"tag": "h2", "class": "singular-sapo"},
        "publish_time_selector": {"tag":"time", "class": "author-time"},
        "author_selector": {"tag": "div", "class": "author-name"}
    }
}
custom_sitemap_or_rss_url = "https://dantri.com.vn/sitemaps/collection-sitemap.xml"

crawler_custom = Crawler(custom_config=custom_site_config, debug=False)
article_metadata_custom = crawler_custom.get_articles(sitemap_url=custom_sitemap_or_rss_url, limit=10)

# Danh sách các bài báo từ sitemap
article_metadata_custom

2025-08-31 17:06:31,219 - Crawler - INFO - Fetching articles via Sitemap (fallback): https://dantri.com.vn/sitemaps/collection-sitemap.xml
2025-08-31 17:06:31,220 - Sitemap - INFO - Fetching sitemap from https://dantri.com.vn/sitemaps/collection-sitemap.xml


[{'url': 'https://dantri.com.vn/event/buyt-tphcm-7076.htm',
  'lastmod': Timestamp('2025-08-30 22:03:21.493000+0700', tz='pytz.FixedOffset(420)')},
 {'url': 'https://dantri.com.vn/event/vong-loai-u23-chau-a-2026-7059.htm',
  'lastmod': Timestamp('2025-08-29 10:45:33.177000+0700', tz='pytz.FixedOffset(420)')},
 {'url': 'https://dantri.com.vn/event/dan-khi-tai-tham-gia-dieu-binh-dieu-hanh-a80-7035.htm',
  'lastmod': Timestamp('2025-08-27 14:05:36.373000+0700', tz='pytz.FixedOffset(420)')},
 {'url': 'https://dantri.com.vn/event/bao-kajiki-con-bao-so-5-6999.htm',
  'lastmod': Timestamp('2025-08-24 06:35:53.313000+0700', tz='pytz.FixedOffset(420)')},
 {'url': 'https://dantri.com.vn/event/cong-an-tphcm-6963.htm',
  'lastmod': Timestamp('2025-08-20 16:57:51.757000+0700', tz='pytz.FixedOffset(420)')},
 {'url': 'https://dantri.com.vn/event/ha-tang-giao-thong-tphcm-6962.htm',
  'lastmod': Timestamp('2025-08-20 16:39:18.257000+0700', tz='pytz.FixedOffset(420)')},
 {'url': 'https://dantri.com.vn/e

In [ ]:
from vnstock_news.core.batch import BatchCrawler

# Fetch articles from a predefined sitemap
crawler = BatchCrawler(custom_config=custom_site_config, debug=False)
articles_df = crawler.fetch_articles(custom_sitemap_or_rss_url, limit=5)
articles_df